In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from matplotlib.image import imread
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import *
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Tensorflow version: 2.3.0
Keras Version: 2.4.0
GPU is NOT AVAILABLE


In [2]:
data_dev = []
data_test = []
data_train = []
file_name_dev = "audioDataDev.pickle"
file_name_test = "audioDataTest.pickle"
file_name_train = "audioDataTrain.pickle"

with (open(file_name_dev, "rb")) as f:
    while True:
        try:
            data_dev.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_test, "rb")) as f:
    while True:
        try:
            data_test.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_train, "rb")) as f:
    while True:
        try:
            data_train.append(pickle.load(f))
        except EOFError:
            break
f.close()

In [3]:
X_dev = []
y_dev = []
X_train = []
y_train = []
X_test = []
y_test = []

for i in range(len(data_dev)-1):
    X_dev.append(data_dev[i+1][3])
for i in range(len(data_dev)-1):
    y_dev.append(data_dev[i+1][6:12])
    
for i in range(len(data_test)-1):
    X_test.append(data_test[i+1][3])
for i in range(len(data_test)-1):
    y_test.append(data_test[i+1][6:12])
    
for i in range(len(data_train)-1):
    X_train.append(data_train[i+1][3])
for i in range(len(data_train)-1):
    y_train.append(data_train[i+1][6:12])

X_dev = np.array(X_dev)
y_dev = np.array(y_dev)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)


In [4]:
y_dev = np.ceil(y_dev)
y_test = np.ceil(y_test)
y_train = np.ceil(y_train)

In [5]:
X_train.shape

(16327, 128, 850)

In [6]:
# get the model
def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=(2), activation='relu', input_shape=(128, 850)))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=64, kernel_size=(5), activation='relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=(5), activation="relu"))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=(5), activation="relu"))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(60, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    return model
 
# evaluate a model using repeated k-fold cross-validation
adam = Adam(0.0005)
model = get_model()
model.compile(optimizer=adam,loss='mean_squared_error',metrics=['MeanSquaredError','AUC','accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_dev, y_dev), batch_size=1000)

Epoch 1/10
17/17 [==============================] - 21s 1s/step - loss: 0.3077 - mean_squared_error: 0.3077 - auc: 0.5802 - accuracy: 0.2588 - val_loss: 0.2667 - val_mean_squared_error: 0.2667 - val_auc: 0.6907 - val_accuracy: 0.6927
Epoch 2/10
17/17 [==============================] - 11s 626ms/step - loss: 0.3067 - mean_squared_error: 0.3067 - auc: 0.6475 - accuracy: 0.4902 - val_loss: 0.2658 - val_mean_squared_error: 0.2658 - val_auc: 0.6928 - val_accuracy: 0.6927
Epoch 3/10
17/17 [==============================] - 10s 564ms/step - loss: 0.3058 - mean_squared_error: 0.3058 - auc: 0.6719 - accuracy: 0.6226 - val_loss: 0.2649 - val_mean_squared_error: 0.2649 - val_auc: 0.6928 - val_accuracy: 0.6927
Epoch 4/10
17/17 [==============================] - 10s 565ms/step - loss: 0.3049 - mean_squared_error: 0.3049 - auc: 0.6802 - accuracy: 0.6634 - val_loss: 0.2640 - val_mean_squared_error: 0.2640 - val_auc: 0.7009 - val_accuracy: 0.6927
Epoch 5/10
17/17 [==============================] - 10s

In [14]:
yhat = model.predict(X_test)

In [28]:
model.save('modelCNN')

INFO:tensorflow:Assets written to: modelCNN\assets


In [15]:
yhat = yhat*3
yhat = yhat.round()
print(yhat[100])
print(y_test[100])

[1. 1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0.]


In [20]:
print(yhat[2])
print(y_test[2])

[1. 1. 0. 0. 0. 0.]
[0. 1. 1. 0. 1. 0.]


In [17]:
results=[]
total = 0
similar = 0
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        if y_test[i][j] == yhat[i][j]:
            similar += 1
        total += 1
acc = similar/total

print('>%.3f' % acc)
results.append(acc)

>0.674
